# Crawler for N+1

In [ ]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import requests
import re
from string import punctuation

%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

import time
from tqdm import tqdm


The URLs to the articles at N+1 have the following structure: the common prefix (https://nplus1.ru/news/), the date, and the uniques name of the article

Firstly, we need to find all the necessary dates. I decided to get the dates from 1.1.2020 to the 28.08.2023. I do not want the conditions for the function to be very complex, therefre I omit those days which do not occur in every month (29-31). Thus, the articles I am going to obtain are those which were released in the first 28 days of each month.

In [ ]:
def validLinks():
  valid_links = []
  for year in range(2020, 2024):
    for month in range(1, 13):
      if year == 2023 and month == 9:
        break
      if month < 10:
        zero = '0'
      else:
        zero = ''
      for day in range(1, 29):
        if day < 10:
          zero_day = '0'
        else:
          zero_day = ''
        valid_link = 'https://nplus1.ru/news/' + str(year) + '/' + zero + str(month) + '/' + zero_day + str(day)
        valid_links.append(valid_link)
  return valid_links

In [ ]:
urls_list = validLinks()

print(len(urls_list))

for l in urls_list[-5:]:
  print(l)

1232
https://nplus1.ru/news/2023/08/24
https://nplus1.ru/news/2023/08/25
https://nplus1.ru/news/2023/08/26
https://nplus1.ru/news/2023/08/27
https://nplus1.ru/news/2023/08/28


The function which gets the affixes for links:

In [ ]:
def getHrefs(link):
    news_url = []
    r = requests.get(link)
    window = re.search('window.materials.+', r.text).group()
    links = re.split(r'url', window)
    for l in links[1:len(links)-3]:
      spl = re.split('/', l)
      if len(spl) <= 7:
        pass
      else:
        cont = spl[7]
        affix = re.split(r'\\u0022', cont)[0]
        news_url.append(link+'/'+affix)
    return news_url

Then I will write the full links down to a new file. It order to not rerun this time-consuming process occasionally, I wrapped it into comments.

In [ ]:
#with open('links.txt', 'w', encoding='utf-8') as f:
#  for lnk in urls_list:
#    for art_link in getHrefs(lnk):
#      print(art_link)
#      f.write(art_link)
#      f.write('\n')
#  f.close()

In [ ]:
#read from the file

with open('links.txt', 'r', encoding='utf-8') as f:
  text = f.read()
  lines = text.splitlines()
  f.close()

In [ ]:
#this is how many links I've obtained

print(len(lines))

10643


Now let's go through every link and wrie the text and the metadata (author, date, time, topic, etc) to a new file. Among others, there is our future target variable — the difficulty of the article.

From that file, a dataframe will be built. I use two tildas '~~' as a separator since otherwise there is a risk that a common separator like comma or tabulation could be found in a text.

In [ ]:
with open('db.txt', 'w+', encoding='utf-8') as f:
  f.write('~~'.join(['time','date','difficulty','topic','heading','author','text']))
  f.write('\n')
  for url in lines:
    print(url)
    session = requests.session()
    response = session.get(url)
    if response.status_code == 200:
      html_content = response.content
      soup = BeautifulSoup(html_content, 'html.parser')
      text_elements = soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'span'])
      scraped_text = ' '.join([element.get_text().replace('\xa0', ' ').lower() for element in text_elements if element])
      strings = scraped_text.split('\n')
      if len(strings) > 7:
        if len(strings[5].split(' ')) == 2:
          author = strings[5]
        else:
          author = ''
        attrs = strings[2].split(' ')
        if len(attrs) > 4:
          row = '~~'.join([attrs[1],attrs[2],attrs[3],attrs[4],
                           ' '.join([w.strip(punctuation) for w in strings[3].split(' ') if w]),
                          author, ' '.join([w.strip(' ').strip(punctuation) for w in strings[6].split(' ')[:-3]]).split('читать дальше')[0].split('\t')[0]])
          if row and len(row.split('~~')) == 7:
            f.write(row)
            f.write('\n')
      else:
        pass
  f.close()

Выходные данные были обрезаны до нескольких последних строк (5000).
https://nplus1.ru/news/2021/09/15/jupiter-fireball
https://nplus1.ru/news/2021/09/15/geriatric-life-extension
https://nplus1.ru/news/2021/09/15/rivian
https://nplus1.ru/news/2021/09/15/quantum-restore
https://nplus1.ru/news/2021/09/15/klebsiella-vaccine
https://nplus1.ru/news/2021/09/15/t-650
https://nplus1.ru/news/2021/09/15/vinland-map
https://nplus1.ru/news/2021/09/15/eciton-hamatum
https://nplus1.ru/news/2021/09/15/neolithic-immunity
https://nplus1.ru/news/2021/09/16/liquid-steering
https://nplus1.ru/news/2021/09/16/prometei
https://nplus1.ru/news/2021/09/16/mozart-epilepsy
https://nplus1.ru/news/2021/09/16/amphibious
https://nplus1.ru/news/2021/09/16/rockettrain
https://nplus1.ru/news/2021/09/16/enlight-short-list-2021
https://nplus1.ru/news/2021/09/16/sn-requiem
https://nplus1.ru/news/2021/09/16/spin-orbit
https://nplus1.ru/news/2021/09/16/nuclear
https://nplus1.ru/news/2021/09/16/medieval-disability
https://nplu

Great! We can get a dataframe.

In [ ]:
df = pd.read_csv('db.txt', sep='~~', engine='python')
df.shape

(10107, 7)

In [ ]:
df.sample(n=10)

,time,date,difficulty,topic,heading,author,text
6047,16:50,26.11.21,2.6,it,нейросеть научили понимать время по часам со с...,григорий копиев,британские разработчики научили нейросеть пон...
8674,14:17,11.01.23,3.5,биология,биологи научили мышей фотосинтезировать,фёдор поляков,китайские биологи внедрили тилакоиды хлоропла...
9437,10:24,12.05.23,1.9,космонавтика,оаэ выбрали астероиды для своей второй межплан...,александр войтюк,объединенные арабские эмираты объявили список...
4606,12:47,24.06.21,2.6,археология,археологи осветили пещеру палеолитическими мет...,винера андреева,с помощью серии экспериментов
6639,13:44,10.02.22,3.1,медицина,подстраивающийся под дыхание кардиостимулятор ...,анастасия кузнецова–фантони,новозеландские исследователи испытали на овца...
7745,16:19,26.07.22,8.1,физика,фотонным временным кристаллам предрекли роль л...,марат хамадеев,израильские физики построили теорию взаимодей...
6018,19:14,24.11.21,6.9,физика,физики применили материально-волновую лупу к к...,марат хамадеев,немецкие физики предложили новый подход к опт...
5140,09:34,18.08.21,4.5,ботаника,генетики реконструировали распространение перц...,сергей коленов,генетики изучили более десяти тысяч образцов ...
6498,22:32,24.01.22,1.7,космонавтика,телескоп «джеймс уэбб» вышел на орбиту вокруг ...,александр войтюк,космический инфракрасный телескоп «джеймс уэб...
5125,10:14,17.08.21,2.5,зоология,новозеландские веснянки утратили крылья по вин...,сергей коленов,после того как маори несколько столетий назад...


In [ ]:
df.isna().sum()

time            0
date            0
difficulty      0
topic           4
heading         0
author        120
text          241
dtype: int64